# 네이버 영화평 감성 분석

In [2]:
!pip install Konlpy > /dev/null

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
#네이버 영화 리뷰 데이터로 검색
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep = "\t")

In [6]:
print(train_df.shape,test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리
- 트레인 데이터셋

In [7]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [8]:
# Null 데이터 제거
train_df.dropna(how='any',inplace=True)
train_df.shape

(149995, 3)

In [10]:
# 중복여부 확인
train_df.document.nunique()

146182

In [11]:
# 중복 데이터 제거
train_df.drop_duplicates(subset = ['document'],inplace=True)
train_df.shape

(146182, 3)

In [12]:
# 긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [13]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [14]:
test_df.dropna(how='any',inplace=True)

In [15]:
test_df.document.nunique()

49157

In [16]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [18]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리
- train dataset

In [21]:
# 한글 이외의 문자는 공백으로 처리하고, strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [22]:
# 만일 후기에 영어로 써 놨으면, 영어가 지워지므로, null data로 됨
# ''만 남은 데이터 --> np.nan으로 대체한 후 제거
train_df.document.replace('',np.nan, inplace=True)
train_df.document.isna().sum()

789

In [23]:
train_df.dropna(how='any',inplace=True)
train_df.shape

(145393, 3)

- 테스트 데이터셋

In [25]:

test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('',np.nan, inplace=True)
test_df.document.isna().sum()
test_df.dropna(how='any',inplace=True)
test_df.shape

(48852, 3)

### 전처리가 끝난 데이터는 저장

In [26]:
train_df.to_csv('naver_movie_train_전처리완료.tsv',sep = '\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv',sep = '\t', index=False)


### 3. 한글 처리

In [28]:
from konlpy.tag import Okt
okt = Okt()

In [29]:
text = "교도소 이야기구먼 솔직히 재미는 없다평점 조정"
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [30]:
okt.morphs(text,stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [31]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text,stem=True) if word not in stopwords])

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [33]:
from tqdm.notebook import tqdm

X_train= []
for sentence in tqdm(train_df.document):
  morphs= okt.morphs(sentence,stem=True)
  tmp_str = ' '.join([word for word in morphs if word not in stopwords])
  X_train.append(tmp_str)


  0%|          | 0/145393 [00:00<?, ?it/s]

In [34]:
X_test= []
for sentence in tqdm(test_df.document):
  morphs= okt.morphs(sentence,stem=True)
  tmp_str = ' '.join([word for word in morphs if word not in stopwords])
  X_test.append(tmp_str)


  0%|          | 0/48852 [00:00<?, ?it/s]

### 4. 피쳐 변환 + 모델 학습/ 평가 

In [37]:
y_train = train_df.label.values
y_test = test_df.label.values

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [38]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([('CVECT',cvect),('LR',lr)])
pipeline.fit(X_train,y_train)

Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [39]:
pipeline.score(X_test,y_test)

0.8273356259723246

### 5. 실제 데이터 테스트


In [41]:
review1 = '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'


In [43]:
#전처리
import re
review1 = re.sub('[^가-힣]', ' ',review1)
review2 = re.sub('[^가-힣]', ' ',review2)

In [44]:
morphs = okt.morphs(review1,stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2,stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])

In [45]:
pipeline.predict([review1, review2])

array([1, 0])

### 6. 최적 파라메터 찾기

In [46]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range':[(1,1),(1,2)],
    'CVECT__max_df':[0.95,0.98],
    'LR__C':[1,5]
}

In [47]:
grid_pipe = GridSearchCV(pipeline,params,scoring='accuracy',cv =3)
%time grid_pipe.fit(X_train,y_train)

CPU times: user 5min 46s, sys: 3min 48s, total: 9min 34s
Wall time: 5min 55s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [48]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [50]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.8479898468844673